In [ ]:
# Install evaluate library for load_metric function
%pip install evaluate

In [ ]:
%pip install --upgrade accelerate

Run on ANLI-2k


In [ ]:
"google/electra-small-discriminator"
"FacebookAI/roberta-base"
"google-bert/bert-base-uncased"

Train on full to get training dynamics

In [38]:
%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/anli-2k" \
    --model_name_or_path "FacebookAI/roberta-base" \
    --output_dir ./data-model/anli-2k/all-data \
    --do_train \
    --do_test \
    --train_logger \
    --num_train_epochs 10 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 256 \
    --val-index-path ./data-model/anli-2k/val_index.npy \
    --seed 42 \
    --data_seed 42

[Errno 2] No such file or directory: 'd2pruning/'
/home/automl/d2pruning
Traceback (most recent call last):
  File "/usr/lib/python3.10/functools.py", line 952, in __set_name__
    def __set_name__(self, owner, name):
KeyboardInterrupt

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/automl/d2pruning/train_nlp_explore.py", line 30, in <module>
    import datasets
  File "/home/automl/d2pruning/venv/lib/python3.10/site-packages/datasets/__init__.py", line 17, in <module>
    from .arrow_dataset import Dataset
  File "/home/automl/d2pruning/venv/lib/python3.10/site-packages/datasets/arrow_dataset.py", line 54, in <module>
    import fsspec
  File "/home/automl/d2pruning/venv/lib/python3.10/site-packages/fsspec/__init__.py", line 9, in <module>
    from .mapping import FSMap, get_mapper
  File "/home/automl/d2pruning/venv/lib/python3.10/site-packages/fsspec/mapping.py", line 13, in <module>
    class FSMap(MutableMappi

Save feature embeddings and importance scores

In [33]:
%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/anli-2k" \
    --model_name_or_path "google-bert/bert-base-uncased" \
    --output_dir ./data-model/anli-2k/all-data \
    --do_eval \
    --eval_train \
    --save_feature \
    --save_confidence \
    --save_importance_scores \
    --training_dynamics \
    --per_device_eval_batch_size 16 \
    --max_length 256 \
    --val-index-path ./data-model/anli-2k/val_index.npy \
    --seed 42 \
    --data_seed 42

[Errno 2] No such file or directory: 'd2pruning/'
/home/automl/d2pruning
06/28/2025 09:49:40 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

Loading validation set from saved index
Reduced training set from 2000 to 1801 for creating validation set
/home/automl/d2pruning/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /home/automl/.cache/huggingface/hub/models--google-bert--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertConfig {
  "_name_or_path": "google-bert/bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,

Prun and train on 50% data

In [34]:
CORESET_RATIO = 0.5
N_NEIGHBOR = 10
GAMMA = 0.1

%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/anli-2k" \
    --model_name_or_path "google-bert/bert-base-uncased" \
    --output_dir ./data-model/anli-2k/coreset-0.5 \
    --do_train \
    --do_test \
    --coreset \
    --coreset-mode class \
    --budget-mode uniform \
    --sampling-mode graph \
    --data-score-path ./data-model/anli-2k/all-data/data-score-goddawg.pickle \
    --feature-path ./data-model/anli-2k/all-data/train-features.npy \
    --coreset-key forgetting \
    --coreset-ratio {CORESET_RATIO} \
    --mis-ratio 0.4 \
    --label-balanced \
    --num_train_epochs 10 \
    --per_device_train_batch_size 32 \
    --learning_rate 2e-5 \
    --max_length 256 \
    --val-index-path ./data-model/anli-2k/val_index.npy \
    --n-neighbor {N_NEIGHBOR} \
    --gamma {GAMMA} \
    --graph-mode sum \
    --graph-sampling-mode weighted \
    --seed 42 \
    --data_seed 42

[Errno 2] No such file or directory: 'd2pruning/'
/home/automl/d2pruning
06/28/2025 09:50:42 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

Loading validation set from saved index
Reduced training set from 2000 to 1801 for creating validation set
Using descending order
Bad data -> High priority accumulated_margin: tensor([-1.4754, -1.4671, -1.4604, -1.3715, -1.3330, -1.3192, -1.3108, -1.3044,
        -1.2968, -1.2751, -1.2328, -1.2296, -1.2239, -1.2204, -1.2138])
Prune 720 samples.
Frequency of bin counts [(344, 1), (707, 1), (30, 1)]
Skipping 0 empty bins in total 3 bins
/home/automl/d2pruning/core/data/Coreset.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rst[sorted_index] = torch.tensor(budgets).type(torch.int)
Using density sampl

Train D(Full data) and D'(50% coreset) on BERT base uncased

In [ ]:
%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/anli-2k" \
    --model_name_or_path bert-base-uncased \
    --output_dir ./data-model/anli-2k/roberta_base/bert/ \
    --do_train \
    --do_test \
    --train_logger \
    --num_train_epochs 10 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 256 \
    --val-index-path ./data-model/anli-2k/val_index.npy

In [37]:
%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/anli-2k" \
    --model_name_or_path "google-bert/bert-base-uncased" \
    --output_dir ./data-model/anli-2k/bert_base/bert-50%/ \
    --do_train \
    --do_test \
    --train_logger \
    --num_train_epochs 10 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 256 \
    --val-index-path ./data-model/anli-2k/val_index.npy \
    --train-index-path ./data-model/anli-2k/coreset-0.5/coreset-goddawg.npy \
    --seed 42 \
    --data_seed 42

[Errno 2] No such file or directory: 'd2pruning/'
/home/automl/d2pruning
06/28/2025 10:00:36 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

Loading validation set from saved index
Reduced training set from 2000 to 1801 for creating validation set
/home/automl/d2pruning/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /home/automl/.cache/huggingface/hub/models--google-bert--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertConfig {
  "_name_or_path": "google-bert/bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,

### Run on AGNews-5K



In [39]:
"google/electra-small-discriminator"
"FacebookAI/roberta-base"
"google-bert/bert-base-uncased"

'google-bert/bert-base-uncased'

In [23]:
%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/agnews-6k" \
    --model_name_or_path "google-bert/bert-base-uncased" \
    --output_dir ./data-model/agnews-6k/all-data \
    --do_train \
    --do_test \
    --train_logger \
    --num_train_epochs 10 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 256 \
    --val-index-path ./data-model/agnews-6k//val_index.npy \
    --seed 42 \
    --data_seed 42

[Errno 2] No such file or directory: 'd2pruning/'
/home/automl/d2pruning
06/28/2025 18:41:15 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

Loading validation set from saved index
Reduced training set from 6000 to 5000 for creating validation set
/home/automl/d2pruning/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /home/automl/.cache/huggingface/hub/models--google-bert--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertConfig {
  "_name_or_path": "google-bert/bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,

In [ ]:
%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/agnews-6k" \
    --model_name_or_path "google/electra-small-discriminator"\
    --output_dir ./data-model/agnews-6k//all-data \
    --do_eval \
    --eval_train \
    --save_feature \
    --save_confidence \
    --save_importance_scores \
    --training_dynamics \
    --per_device_eval_batch_size 16 \
    --max_length 256 \
    --val-index-path ./data-model/agnews-6k/val_index.npy \
    --seed 42 \
    --data_seed 42

[Errno 2] No such file or directory: 'd2pruning/'
/home/automl/d2pruning
06/28/2025 18:39:30 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

Loading validation set from saved index
Reduced training set from 6000 to 5000 for creating validation set
/home/automl/d2pruning/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /home/automl/.cache/huggingface/hub/models--google--electra-small-discriminator/snapshots/fa8239aadc095e9164941d05878b98afe9b953c3/config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_

In [16]:
CORESET_RATIO = 0.5
N_NEIGHBOR = 10
GAMMA = 0.1

%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/agnews-6k" \
    --model_name_or_path "google/electra-small-discriminator" \
    --output_dir ./data-model/agnews-6k//coreset-0.5 \
    --do_train \
    --do_test \
    --coreset \
    --coreset-mode class \
    --budget-mode uniform \
    --sampling-mode graph \
    --data-score-path ./data-model/agnews-6k/all-data/data-score-goddawg.pickle \
    --feature-path ./data-model/agnews-6k//all-data/train-features.npy \
    --coreset-key forgetting \
    --coreset-ratio {CORESET_RATIO} \
    --mis-ratio 0.4 \
    --label-balanced \
    --num_train_epochs 10 \
    --per_device_train_batch_size 32 \
    --learning_rate 2e-5 \
    --max_length 256 \
    --val-index-path ./data-model/agnews-6k/val_index.npy \
    --n-neighbor {N_NEIGHBOR} \
    --gamma {GAMMA} \
    --graph-mode sum \
    --graph-sampling-mode weighted \
    --seed 42 \
    --data_seed 42

[Errno 2] No such file or directory: 'd2pruning/'
/home/automl/d2pruning


/home/automl/d2pruning/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


06/28/2025 17:40:17 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

Loading validation set from saved index
Reduced training set from 6000 to 5000 for creating validation set
Using descending order
Bad data -> High priority accumulated_margin: tensor([-9.8320, -8.8350, -8.3520, -6.6949, -6.4507, -6.0567, -5.7893, -5.6004,
        -5.4415, -5.3974, -3.9379, -3.7875, -3.3515, -3.2190, -3.0092])
Prune 2000 samples.
Frequency of bin counts [(838, 1), (928, 1), (643, 1), (591, 1)]
Skipping 0 empty bins in total 4 bins
/home/automl/d2pruning/core/data/Coreset.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rst[sorted_index] = torch.tensor(budgets).type(torch.int)
Using density sampling...
Starting process for label   0 with 838 samples
8380 c

In [19]:
%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/agnews-6k" \
    --model_name_or_path "FacebookAI/roberta-base" \
    --output_dir ./data-model/agnews-6k/electra/bert-50%/ \
    --do_train \
    --do_test \
    --train_logger \
    --num_train_epochs 10 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 256 \
    --val-index-path ./data-model/agnews-6k/val_index.npy \
    --train-index-path ./data-model/agnews-6k//coreset-0.5/coreset-goddawg.npy \
    --seed 42 \
    --data_seed 42

[Errno 2] No such file or directory: 'd2pruning/'
/home/automl/d2pruning


06/28/2025 17:51:33 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

Loading validation set from saved index
Reduced training set from 6000 to 5000 for creating validation set
/home/automl/d2pruning/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /home/automl/.cache/huggingface/hub/models--FacebookAI--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b/config.json
Model config RobertaConfig {
  "_name_or_path": "FacebookAI/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "fin

In [ ]:
"google/electra-small-discriminator"
"FacebookAI/roberta-base"
"google-bert/bert-base-uncased"

### Run on AGNews-5K



In [5]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install datasets

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
Using cached datasets-3.6.0-py3-none-any.whl (491 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 1.5 MB/s eta 0:00:00a 0:00:01
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.3.2-py2.py3-none-any.whl (7.6 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 8.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: fsspec━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/15 [multidict]
    Found existing inst

In [3]:
"google/electra-small-discriminator"
"FacebookAI/roberta-base"
"google-bert/bert-base-uncased"

'google-bert/bert-base-uncased'

In [12]:
%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/agnews-6k" \
    --model_name_or_path "google-bert/bert-base-uncased" \
    --output_dir ./data-model/agnews-6k/all-data \
    --do_train \
    --do_test \
    --train_logger \
    --num_train_epochs 10 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 256 \
    --val-index-path ./data-model/agnews-6k//val_index.npy \
    --seed 42 \
    --data_seed 42

[Errno 2] No such file or directory: 'd2pruning/'
/drive1/nammt/d2pruning
06/28/2025 16:49:36 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

Loading validation set from saved index
Reduced training set from 6000 to 5000 for creating validation set
/drive1/nammt/d2pruning/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /home/automl/.cache/huggingface/hub/models--google-bert--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertConfig {
  "_name_or_path": "google-bert/bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0